In [1]:
import pandas as pd
import bz2
import glob
from sklearn.utils import shuffle
import spacy
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV




In [20]:
train_data_no = pd.read_csv('train_data_no.csv')
val_data_no = pd.read_csv('val_data_no.csv')
test_data_no = pd.read_csv('test_data_no.csv')

train_data_alpha = pd.read_csv('train_data_alpha.csv')
val_data_alpha = pd.read_csv('val_data_alpha.csv')
test_data_alpha = pd.read_csv('test_data_alpha.csv')

train_data_lemma = pd.read_csv('train_data_lemma.csv')
val_data_lemma = pd.read_csv('val_data_lemma.csv')
test_data_lemma = pd.read_csv('test_data_lemma.csv')


In [ ]:
def train_pipeline(train_data_, test_data_):
    pipe = Pipeline([('vectorizer', TfidfVectorizer(lowercase = False)), ('classifier', LogisticRegression())])
    pipe.fit(train_data_['text'], train_data_['label'])

    pred_ = pipe.predict(test_data_['text'])
    print(classification_report(test_data_['label'], pred_, digits=3))


train_pipeline(train_data_no, val_data_noo)
train_pipeline(train_data_alpha, val_data_alph)
train_pipeline(train_data_lemma, val_data_le)

In [32]:

parameters = {'vectorizer__binary':[True, False],
              'vectorizer__ngram_range':[(1, 1), (1, 2), (2,2), (1, 3), (2, 3),(3, 3)],
             }


pipe = Pipeline([('vectorizer', TfidfVectorizer(lowercase = False)), ('classifier', LogisticRegression())])
clf = GridSearchCV(pipe, parameters, verbose=3)
clf.fit(train_data_alpha['text'], train_data_alpha['label'])

print(sorted(clf.cv_results_.keys()))

print(clf.cv_results_['mean_test_score'])
best_params = clf.cv_results_['params'][clf.cv_results_['rank_test_score'][0]-1]
print(best_params)
print(clf.cv_results_['rank_test_score'])


Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] vectorizer__binary=True, vectorizer__ngram_range=(1, 1) .........
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV]  vectorizer__binary=True, vectorizer__ngram_range=(1, 1), score=0.894, total=   6.0s
[CV] vectorizer__binary=True, vectorizer__ngram_range=(1, 1) .........
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.9s remaining:    0.0s
[CV]  vectorizer__binary=True, vectorizer__ngram_range=(1, 1), score=0.892, total=   5.6s
[CV] vectorizer__binary=True, vectorizer__ngram_range=(1, 1) .........
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   11.5s remaining:    0.0s
[CV]  vectorizer__binary=True, vectorizer__ngram_range=(1, 1), score=0.889, total=   5.7s
[CV] vectorizer__binary=True, vectorizer__ngram_range=(1, 1) .........
[CV]  vectorizer__binary=True, vectorizer__ngram_range=(1, 1), score=0.890, total=   6.1s
[CV] vectorizer__binary=True, vectorizer__ngram_range=

In [ ]:
pipe = Pipeline([('vectorizer', TfidfVectorizer(
                                                ngram_range = best_params['vectorizer__ngram_range'],
                                                binary = best_params['vectorizer__binary'])
                                                ), 
                  ('classifier', LogisticRegression())
                ])


pipe.fit(train_data_alpha['text'], train_data_alpha['label'])

pred = pipe.predict(test_data_alpha['text'])
print(classification_report(test_data_alpha['label'], pred, digits=3))